# 🏥 Análisis Geoespacial de Hospitales en Perú
## Notebook 1: Procesamiento y Limpieza de Datos

### 📋 Objetivos de este notebook:
- ✅ Configurar el ambiente de trabajo
- ✅ Crear datos de ejemplo de hospitales
- ✅ Convertir a formato geoespacial (GeoDataFrame)
- ✅ Realizar mapas básicos de verificación
- ✅ Preparar datos para análisis posteriores

### 📊 Fuentes de datos:
- **MINSA-IPRESS**: Hospitales públicos operativos
- **INEI**: Centros poblados
- **Shapefile**: Distritos del Perú

---

In [20]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import folium
from shapely.geometry import Point

print("Librerías importadas correctamente")

Librerías importadas correctamente


---
## 📥 1. Carga de Datos Reales MINSA-IPRESS

Ahora trabajaremos con el dataset oficial del MINSA con **20,819 establecimientos** de salud.

### 📊 Dataset oficial incluye:
- **Hospitales y centros de salud** de todo el Perú
- **Coordenadas geográficas** (NORTE, ESTE en UTM)
- **Información administrativa** completa
- **Estado operativo** de cada establecimiento

In [21]:
# 📥 CARGAR DATOS REALES DEL MINSA
import os

def cargar_datos_ipress():
    """Carga y explora el dataset oficial MINSA-IPRESS"""
    
    # Ruta correcta: desde notebooks hacia data
    archivo_csv = "../data/ipress_hospitales.csv"
    
    if not os.path.exists(archivo_csv):
        print(f"❌ Archivo no encontrado: {archivo_csv}")
        print("📁 Verifica que el archivo esté en la carpeta 'data'")
        return None
    
    print("📥 Cargando datos MINSA-IPRESS...")
    
    # Leer CSV (codificación ISO-8859-1 según el archivo)
    df = pd.read_csv(archivo_csv, encoding='iso-8859-1')
    
    print(f"✅ Datos cargados: {len(df):,} establecimientos")
    print(f"📊 Columnas: {len(df.columns)}")
    
    return df

# Cargar datos
df_ipress = cargar_datos_ipress()

if df_ipress is not None:
    print("\n🔍 Columnas disponibles:")
    print(list(df_ipress.columns))
    
    print("\n🔍 Primeras 3 filas:")
    display(df_ipress.head(3))
    
    print("\n📊 Información del dataset:")
    print(df_ipress.info())

📥 Cargando datos MINSA-IPRESS...
✅ Datos cargados: 20,819 establecimientos
📊 Columnas: 33

🔍 Columnas disponibles:
['Institución', 'Código Único', 'Nombre del establecimiento', 'Clasificación', 'Tipo', 'Departamento', 'Provincia', 'Distrito', 'UBIGEO', 'Dirección', 'Código DISA', 'Código Red', 'Código Microrred', 'DISA', 'Red', 'Microrred', 'Código UE', 'Unidad Ejecutora', 'Categoria', 'Teléfono', 'Tipo Doc.Categorización', 'Nro.Doc.Categorización', 'Horario', 'Inicio de Actividad', 'Director Médico y/o Responsable de la Atención de Salud', 'Estado', 'Situación', 'Condición', 'Inspección', 'NORTE', 'ESTE', 'COTA', 'CAMAS']

🔍 Primeras 3 filas:


,Institución,Código Único,Nombre del establecimiento,Clasificación,Tipo,Departamento,Provincia,Distrito,UBIGEO,Dirección,...,Inicio de Actividad,Director Médico y/o Responsable de la Atención de Salud,Estado,Situación,Condición,Inspección,NORTE,ESTE,COTA,CAMAS
0,PRIVADO,16618,SONRIE MAS,CONSULTORIOS MEDICOS Y DE OTROS PROFESIONALES ...,ESTABLECIMIENTO DE SALUD SIN INTERNAMIENTO,LIMA,LIMA,SANTIAGO DE SURCO,150140,"AV. CAMINOS DEL INCA 2028 OF.201, URBANIZACION...",...,11/03/2013,GILBERTO MARTIN AQUIJE GAVILAN,ACTIVADO,,EN FUNCIONAMIENTO,,NaN,NaN,NaN,NaN
1,GOBIERNO REGIONAL,7050,AMBATO,PUESTOS DE SALUD O POSTAS DE SALUD,ESTABLECIMIENTO DE SALUD SIN INTERNAMIENTO,CAJAMARCA,CUTERVO,SANTA CRUZ,60611,COMUNIDAD DE AMBATO,...,29/06/2006,IDELSO MENOR CHAVEZ,ACTIVADO,,EN FUNCIONAMIENTO,,-78.858380,-6.133523,1724.0,NaN
2,GOBIERNO REGIONAL,99,SANTA ISABEL DE YUMBATURO,PUESTOS DE SALUD O POSTAS DE SALUD,ESTABLECIMIENTO DE SALUD SIN INTERNAMIENTO,LORETO,LORETO,PARINARI,160302,ACTUALIZAR,...,01/01/1900,JULIO TONY CAITIMARI MACHOA,ACTIVADO,,EN FUNCIONAMIENTO,,-74.258139,-4.581509,124.0,NaN



📊 Información del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20819 entries, 0 to 20818
Data columns (total 33 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0   Institución                                              20819 non-null  object 
 1   Código Único                                             20819 non-null  int64  
 2   Nombre del establecimiento                               20819 non-null  object 
 3   Clasificación                                            20809 non-null  object 
 4   Tipo                                                     20819 non-null  object 
 5   Departamento                                             20819 non-null  object 
 6   Provincia                                                20819 non-null  object 
 7   Distrito                                                 20819 non-null  object 
 8 

---
## 🔍 2. Exploración de Datos IPRESS

Ahora que tenemos los **20,819 establecimientos** cargados, vamos a explorar la estructura y contenido del dataset para entender mejor qué tipos de establecimientos tenemos.

### 📊 Análisis a realizar:
- Tipos de establecimientos disponibles
- Estados operativos
- Distribución geográfica
- Disponibilidad de coordenadas

In [22]:
# 🔍 EXPLORACIÓN DETALLADA DE DATOS REALES
print("🔍 EXPLORACIÓN DE DATOS IPRESS")
print("=" * 50)

# Información básica
print(f"📊 Total de registros: {len(df_ipress):,}")
print(f"📋 Total de columnas: {len(df_ipress.columns)}")

print("\n🏥 Tipos de establecimientos:")
tipos_count = df_ipress['Tipo'].value_counts().head(10)
for tipo, count in tipos_count.items():
    print(f"  • {tipo}: {count:,}")

print("\n🏛️ Estados operativos:")
estados_count = df_ipress['Estado'].value_counts()
for estado, count in estados_count.items():
    print(f"  • {estado}: {count:,}")

print("\n🌍 Departamentos con más establecimientos:")
dept_count = df_ipress['Departamento'].value_counts().head(10)
for dept, count in dept_count.items():
    print(f"  • {dept}: {count:,}")

# Verificar coordenadas
print("\n📍 Información de coordenadas:")
print(f"  • NORTE válidos: {df_ipress['NORTE'].notna().sum():,}")
print(f"  • ESTE válidos: {df_ipress['ESTE'].notna().sum():,}")

🔍 EXPLORACIÓN DE DATOS IPRESS
📊 Total de registros: 20,819
📋 Total de columnas: 33

🏥 Tipos de establecimientos:
  • ESTABLECIMIENTO DE SALUD SIN INTERNAMIENTO: 17,746
  • SERVICIO MÉDICO DE APOYO: 1,979
  • ESTABLECIMIENTO DE SALUD CON INTERNAMIENTO: 1,094

🏛️ Estados operativos:
  • ACTIVADO: 20,819

🌍 Departamentos con más establecimientos:
  • LIMA: 7,423
  • CAJAMARCA: 1,097
  • PIURA: 1,055
  • AREQUIPA: 1,009
  • JUNIN: 956
  • CUSCO: 861
  • CALLAO: 770
  • LA LIBERTAD: 677
  • LAMBAYEQUE: 673
  • PUNO: 614

📍 Información de coordenadas:
  • NORTE válidos: 7,956
  • ESTE válidos: 7,956


In [23]:
# 🔍 EXPLORAR VALORES EN CLASIFICACIÓN
print("🔍 EXPLORANDO CLASIFICACIONES DISPONIBLES")
print("=" * 50)

# Ver todas las clasificaciones únicas
print("📋 Valores únicos en 'Clasificación':")
clasificaciones = df_ipress['Clasificación'].value_counts()
for clasif, count in clasificaciones.items():
    print(f"  • {clasif}: {count:,}")

# Ver clasificaciones en establecimientos con internamiento
establecimientos_internamiento = df_ipress[df_ipress['Tipo'] == 'ESTABLECIMIENTO DE SALUD CON INTERNAMIENTO']
print(f"\n🏥 Clasificaciones en establecimientos CON INTERNAMIENTO ({len(establecimientos_internamiento):,}):")
clasif_internamiento = establecimientos_internamiento['Clasificación'].value_counts()
for clasif, count in clasif_internamiento.items():
    print(f"  • {clasif}: {count}")

# Buscar patrones con "HOSPITAL"
print("\n🔍 Clasificaciones que contienen 'HOSPITAL':")
mask_hospital_clasif = df_ipress['Clasificación'].str.contains('HOSPITAL', case=False, na=False)
if mask_hospital_clasif.sum() > 0:
    hospital_clasifs = df_ipress[mask_hospital_clasif]['Clasificación'].value_counts()
    for clasif, count in hospital_clasifs.items():
        print(f"  • {clasif}: {count}")
else:
    print("  No se encontraron clasificaciones con 'HOSPITAL'")

🔍 EXPLORANDO CLASIFICACIONES DISPONIBLES
📋 Valores únicos en 'Clasificación':
  • CONSULTORIOS MEDICOS Y DE OTROS PROFESIONALES DE LA SALUD: 7,196
  • PUESTOS DE SALUD O POSTAS DE SALUD: 6,709
  • CENTROS DE SALUD O CENTROS MEDICOS: 1,850
  • POLICLINICOS: 1,040
  • CENTRO ODONTOLOGICO: 611
  • PATOLOGIA CLINICA: 492
  • CENTROS DE SALUD CON CAMAS DE INTERNAMIENTO: 456
  • HOSPITALES O CLINICAS DE ATENCION GENERAL: 368
  • CENTROS MEDICOS ESPECIALIZADOS: 318
  • CENTROS OPTICOS: 317
  • HOSPITALES O CLINICAS DE ATENCION ESPECIALIZADA: 243
  • DIAGNOSTICO POR IMAGENES: 241
  • HEMODIALISIS: 159
  • REHABILITACION: 128
  • SERVICIO DE TRASLADO DE PACIENTES: 108
  • MEDICINA FISICA,REHABILITACION: 57
  • CENTROS DE ATENCION PARA DEPENDIENTES A SUSTANCIAS PSICOACTIVAS Y OTRAS DEPENDENCIAS: 43
  • CENTROS DE MEDICINA ALTERNATIVA: 42
  • PATOLOGIA CLINICA,ANATOMIA PATOLOGICA: 41
  • MEDICINA FISICA: 37
  • ANATOMIA PATOLOGICA: 36
  • ESTABLECIMIENTOS DE RECUPERACION O REPOSO: 31
  • LABORATO

---
## 🏥 3. Filtrado de Hospitales Públicos Operativos

Del total de establecimientos, vamos a filtrar solo los **hospitales públicos que están funcionando** y tienen coordenadas válidas.

### 🎯 Filtros a aplicar:
1. **Tipo**: Solo "HOSPITAL"
2. **Estado**: Solo "FUNCIONANDO" o "ACTIVO"  
3. **Institución**: Solo establecimientos públicos (GOBIERNO)
4. **Coordenadas**: Solo con NORTE y ESTE válidos

Este filtrado nos dará el dataset final para nuestro análisis geoespacial.

In [24]:
# 🏥 FILTROS SIMPLIFICADOS PARA HOSPITALES
def filtrar_hospitales_simplificado(df):
    """Filtros simplificados enfocándose en clasificación de hospitales"""
    
    print("🏥 FILTROS SIMPLIFICADOS DE HOSPITALES")
    print("=" * 40)
    
    df_filtrado = df.copy()
    inicial = len(df_filtrado)
    print(f"📊 Registros iniciales: {inicial:,}")
    
    # 1. Solo clasificaciones de hospitales (esto ya implica internamiento)
    clasificaciones_hospital = [
        'HOSPITALES O CLINICAS DE ATENCION GENERAL',
        'HOSPITALES O CLINICAS DE ATENCION ESPECIALIZADA'
    ]
    mask_clasificacion = df_filtrado['Clasificación'].isin(clasificaciones_hospital)
    df_filtrado = df_filtrado[mask_clasificacion]
    print(f"🏥 Solo hospitales: {len(df_filtrado):,}")
    
    # 2. Solo en funcionamiento
    mask_funcionamiento = df_filtrado['Condición'] == 'EN FUNCIONAMIENTO'
    df_filtrado = df_filtrado[mask_funcionamiento]
    print(f"✅ Solo en funcionamiento: {len(df_filtrado):,}")
    
    # 3. Solo instituciones públicas
    instituciones_publicas = ['GOBIERNO REGIONAL', 'MINSA', 'ESSALUD']
    mask_publico = df_filtrado['Institución'].isin(instituciones_publicas)
    df_filtrado = df_filtrado[mask_publico]
    print(f"🏛️ Solo públicos: {len(df_filtrado):,}")
    
    # 4. Solo con coordenadas válidas
    mask_coords = (
        df_filtrado['NORTE'].notna() & 
        df_filtrado['ESTE'].notna() &
        (df_filtrado['NORTE'] != 0) & 
        (df_filtrado['ESTE'] != 0)
    )
    df_filtrado = df_filtrado[mask_coords]
    print(f"🌍 Con coordenadas válidas: {len(df_filtrado):,}")
    
    print(f"\n📈 Reducción: {inicial:,} → {len(df_filtrado):,} ({len(df_filtrado)/inicial*100:.1f}%)")
    
    return df_filtrado

# Aplicar filtros simplificados
df_hospitales = filtrar_hospitales_simplificado(df_ipress)

if len(df_hospitales) > 0:
    print("\n🏥 Tipos de hospitales:")
    clasif_counts = df_hospitales['Clasificación'].value_counts()
    for clasif, count in clasif_counts.items():
        print(f"  • {clasif}: {count}")
    
    print("\n🏛️ Hospitales por departamento:")
    dept_counts = df_hospitales['Departamento'].value_counts().head(10)
    for dept, count in dept_counts.items():
        print(f"  • {dept}: {count}")
    
    print("\n🏥 Hospitales por institución:")
    inst_counts = df_hospitales['Institución'].value_counts()
    for inst, count in inst_counts.items():
        print(f"  • {inst}: {count}")
    
    print("\n🔍 Muestra final de hospitales:")
    display(df_hospitales[['Nombre del establecimiento', 'Departamento', 'Institución', 'Clasificación']].head())
else:
    print("❌ No se encontraron hospitales con estos filtros")

🏥 FILTROS SIMPLIFICADOS DE HOSPITALES
📊 Registros iniciales: 20,819
🏥 Solo hospitales: 611
✅ Solo en funcionamiento: 608
🏛️ Solo públicos: 250
🌍 Con coordenadas válidas: 232

📈 Reducción: 20,819 → 232 (1.1%)

🏥 Tipos de hospitales:
  • HOSPITALES O CLINICAS DE ATENCION GENERAL: 180
  • HOSPITALES O CLINICAS DE ATENCION ESPECIALIZADA: 52

🏛️ Hospitales por departamento:
  • LIMA: 37
  • LA LIBERTAD: 31
  • ANCASH: 14
  • PUNO: 13
  • JUNIN: 12
  • AREQUIPA: 10
  • SAN MARTIN: 10
  • CAJAMARCA: 10
  • AYACUCHO: 10
  • CUSCO: 9

🏥 Hospitales por institución:
  • GOBIERNO REGIONAL: 144
  • ESSALUD: 69
  • MINSA: 19

🔍 Muestra final de hospitales:


,Nombre del establecimiento,Departamento,Institución,Clasificación
17,CARLOS CORNEJO ROSELLO,PUNO,GOBIERNO REGIONAL,HOSPITALES O CLINICAS DE ATENCION GENERAL
35,CARHUAZ,ANCASH,GOBIERNO REGIONAL,HOSPITALES O CLINICAS DE ATENCION GENERAL
52,"HOSPITAL PROVINCIAL ASCOPE ""ROSA SANCHEZ DE SA...",LA LIBERTAD,GOBIERNO REGIONAL,HOSPITALES O CLINICAS DE ATENCION GENERAL
275,HOSPITAL MARIA AUXILIADORA,LIMA,MINSA,HOSPITALES O CLINICAS DE ATENCION ESPECIALIZADA
339,HOSPITAL I ALBRECHT,LA LIBERTAD,ESSALUD,HOSPITALES O CLINICAS DE ATENCION GENERAL


---
## 🌍 4. Procesamiento de Coordenadas Geográficas

Tenemos **232 hospitales públicos** válidos. Al revisar las coordenadas ESTE/NORTE, descubrimos que ya están en formato geográfico (latitud/longitud).

### 📍 Situación real de coordenadas:
- **ESTE**: Corresponde a **longitud** (grados decimales)
- **NORTE**: Corresponde a **latitud** (grados decimales) 
- **Formato actual**: Ya en EPSG:4326 (WGS84)
- **Acción necesaria**: Solo reasignar columnas correctamente

Este es un hallazgo importante que simplifica nuestro procesamiento.

In [25]:
# 🌍 CORREGIR COORDENADAS INTERCAMBIADAS
def procesar_coordenadas_corregido_v2(df):
    """Corregir asignación: NORTE=longitud, ESTE=latitud"""
    
    print("🌍 CORRIGIENDO COORDENADAS INTERCAMBIADAS...")
    print("=" * 45)
    
    df_coords = df.copy()
    
    # Verificar rango original
    este_min, este_max = df_coords['ESTE'].min(), df_coords['ESTE'].max()
    norte_min, norte_max = df_coords['NORTE'].min(), df_coords['NORTE'].max()
    
    print(f"📊 Coordenadas originales (intercambiadas):")
    print(f"  • ESTE (en realidad latitud): {este_min:.1f} - {este_max:.1f}")
    print(f"  • NORTE (en realidad longitud): {norte_min:.1f} - {norte_max:.1f}")
    
    # Asignación correcta (intercambiada)
    df_coords['latitud'] = df_coords['ESTE']    # ESTE contiene latitudes
    df_coords['longitud'] = df_coords['NORTE']  # NORTE contiene longitudes
    
    print("🔧 Asignación corregida:")
    print("  • latitud ← ESTE")
    print("  • longitud ← NORTE")
    
    # Verificar rango corregido
    lat_min, lat_max = df_coords['latitud'].min(), df_coords['latitud'].max()
    lon_min, lon_max = df_coords['longitud'].min(), df_coords['longitud'].max()
    
    print(f"\n📍 Coordenadas corregidas:")
    print(f"  • Latitud: {lat_min:.1f} - {lat_max:.1f}")
    print(f"  • Longitud: {lon_min:.1f} - {lon_max:.1f}")
    
    # Verificar que estén en rango válido para Perú
    mask_peru = (
        df_coords['latitud'].between(-18.5, 0) &
        df_coords['longitud'].between(-81.5, -68)
    )
    
    coords_validas = mask_peru.sum()
    coords_invalidas = len(df_coords) - coords_validas
    
    print(f"\n✅ Verificación geográfica:")
    print(f"  • Válidas para Perú: {coords_validas}")
    print(f"  • Fuera de rango: {coords_invalidas}")
    
    if coords_invalidas > 0:
        print("🔧 Aplicando filtro geográfico...")
        df_coords = df_coords[mask_peru]
    
    print(f"📍 Dataset final: {len(df_coords)} hospitales")
    
    return df_coords

# Procesar con corrección
df_hospitales_coords = procesar_coordenadas_corregido_v2(df_hospitales)

print("\n🔍 Muestra de coordenadas corregidas:")
display(df_hospitales_coords[['Nombre del establecimiento', 'Departamento', 
                              'latitud', 'longitud']].head())

🌍 CORRIGIENDO COORDENADAS INTERCAMBIADAS...
📊 Coordenadas originales (intercambiadas):
  • ESTE (en realidad latitud): -18.0 - -3.6
  • NORTE (en realidad longitud): -81.3 - -69.1
🔧 Asignación corregida:
  • latitud ← ESTE
  • longitud ← NORTE

📍 Coordenadas corregidas:
  • Latitud: -18.0 - -3.6
  • Longitud: -81.3 - -69.1

✅ Verificación geográfica:
  • Válidas para Perú: 232
  • Fuera de rango: 0
📍 Dataset final: 232 hospitales

🔍 Muestra de coordenadas corregidas:


,Nombre del establecimiento,Departamento,latitud,longitud
17,CARLOS CORNEJO ROSELLO,PUNO,-14.904143,-70.199339
35,CARHUAZ,ANCASH,-9.277484,-77.649385
52,"HOSPITAL PROVINCIAL ASCOPE ""ROSA SANCHEZ DE SA...",LA LIBERTAD,-7.714237,-79.101259
275,HOSPITAL MARIA AUXILIADORA,LIMA,-12.160577,-76.959102
339,HOSPITAL I ALBRECHT,LA LIBERTAD,-8.111071,-79.034930


---
## 🗺️ 5. Creación del GeoDataFrame Final

Con las coordenadas corregidas y validadas, ahora crearemos el **GeoDataFrame** final que será la base para todos nuestros análisis geoespaciales posteriores.

### 📍 Proceso de creación:
- **Geometría**: Puntos creados a partir de longitud/latitud
- **CRS**: EPSG:4326 (WGS84) - estándar mundial
- **Datos**: 232 hospitales públicos en funcionamiento
- **Guardado**: Formato GeoJSON para reutilización

### 💾 Archivo de salida:
El GeoDataFrame se guardará como `hospitales_procesados.geojson` en la carpeta `data/` para usar en los siguientes notebooks de análisis estático, proximidad y mapas interactivos.

In [26]:
# 🗺️ CREAR GEODATAFRAME FINAL
def crear_geodataframe_final(df):
    """Crea GeoDataFrame final con coordenadas corregidas"""
    
    print("🗺️ CREANDO GEODATAFRAME FINAL...")
    print("=" * 30)
    
    # Crear geometría de puntos
    from shapely.geometry import Point
    geometry = [Point(lon, lat) for lon, lat in zip(df['longitud'], df['latitud'])]
    
    # Crear GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')
    
    print(f"✅ GeoDataFrame creado: {len(gdf)} hospitales")
    print(f"🌍 CRS: {gdf.crs}")
    print(f"🏛️ Cobertura: {gdf['Departamento'].nunique()} departamentos")
    
    return gdf

# Crear GeoDataFrame final
gdf_hospitales_final = crear_geodataframe_final(df_hospitales_coords)

# Guardar datos procesados
ruta_salida = '../data/hospitales_procesados.geojson'
gdf_hospitales_final.to_file(ruta_salida, driver='GeoJSON')
print(f"\n💾 Datos guardados en: {ruta_salida}")

print("\n📊 RESUMEN FINAL DEL NOTEBOOK:")
print("=" * 40)
print(f"✅ Total hospitales procesados: {len(gdf_hospitales_final)}")
print(f"🏛️ Departamentos cubiertos: {gdf_hospitales_final['Departamento'].nunique()}")
print(f"🏥 Tipos de hospitales:")
for tipo, count in gdf_hospitales_final['Clasificación'].value_counts().items():
    print(f"   • {tipo}: {count}")
print(f"🏛️ Instituciones:")
for inst, count in gdf_hospitales_final['Institución'].value_counts().items():
    print(f"   • {inst}: {count}")

🗺️ CREANDO GEODATAFRAME FINAL...
✅ GeoDataFrame creado: 232 hospitales
🌍 CRS: EPSG:4326
🏛️ Cobertura: 25 departamentos

💾 Datos guardados en: ../data/hospitales_procesados.geojson

📊 RESUMEN FINAL DEL NOTEBOOK:
✅ Total hospitales procesados: 232
🏛️ Departamentos cubiertos: 25
🏥 Tipos de hospitales:
   • HOSPITALES O CLINICAS DE ATENCION GENERAL: 180
   • HOSPITALES O CLINICAS DE ATENCION ESPECIALIZADA: 52
🏛️ Instituciones:
   • GOBIERNO REGIONAL: 144
   • ESSALUD: 69
   • MINSA: 19


---
## ✅ Notebook 1 Completado Exitosamente

### 🎉 Procesamiento de Datos MINSA-IPRESS Finalizado

#### 📊 Resultados obtenidos:
- ✅ **232 hospitales públicos** validados y procesados
- ✅ Coordenadas corregidas (NORTE/ESTE intercambiados)
- ✅ GeoDataFrame creado con CRS EPSG:4326
- ✅ Datos guardados en `../data/hospitales_procesados.geojson`

#### 🏥 Dataset final incluye:
- **180 hospitales generales** + **52 especializados**
- **Cobertura nacional**: múltiples departamentos
- **Solo instituciones públicas**: GOBIERNO REGIONAL, MINSA, ESSALUD
- **Solo hospitales en funcionamiento**

#### 🚀 Próximos notebooks:
1. **02_static_maps.ipynb**: Mapas estáticos y análisis departamental
2. **03_proximity_analysis.ipynb**: Análisis de proximidad Lima/Loreto
3. **04_interactive_maps.ipynb**: Mapas interactivos con Folium
4. **05_streamlit_app.ipynb**: Dashboard final

¡Base de datos lista para análisis geoespacial completo!